# TORCH.NN
---
> 신경망은 데이터에 대한 연산을 수행하는 계층(layer)/모듈(module)로 구성되어 있습니다.  
> torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공합니다.  
> PyTorch의 모든 모듈은 nn.Module 의 하위 클래스(subclass) 입니다.   
> 신경망은 다른 모듈(계층; layer)로 구성된 모듈입니다. 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있습니다.  


### 1. nn.Module
---
nn.Module는 모든 신경망 모듈의 기본 클래스이다.

모든 모델은 이 클래스의 하위 클래스로 지정해야 한다. 

In [1]:
# 사용 예시
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.relu(self.conv2(x))

### 2. Dezero와의 연관성
---

사실 Dezero에서 Module을 찾아볼 순 없었다.

그래서 직접 코드를 뜯어본 결과

> 코드 : https://pytorch.org/docs/stable/generated/torch.nn.Module.html

가장 유사한 것이 Dezero의 Layer 클래스이다.


In [ ]:
class Layer:
    def __init__(self):
        self._params = set()

    def __setattr__(self, name, value): # object에 존재하는 속성의 값을 바꾸거나 새로운 속성을 생성할때마다 호출되는 특수 메서드
        if isinstance(value, (Parameter, Layer)):
            self._params.add(name)
        super().__setattr__(name, value)

    def __call__(self, *inputs): #함수 형태로 불리는 순간 forward 실행
        outputs = self.forward(*inputs) # 입력값을 순전파로 받아서 출력값 생성
        if not isinstance(outputs, tuple):
            outputs = (outputs, )
        self.inputs = [weakref.ref(x) for x in inputs]
        self.outputs = [weakref.ref(y) for y in outputs]
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self,inputs): #순전파는 이 클래스를 상속받은 유도 클래스에서 정의해야함
        raise NotImplementedError()

    def params(self): # 파라미터를 반환함
        for name in self._params:
            obj = self.__dict__[name]

            if isinstance(obj, Layer):
                yield from obj.params()
            else:
                yield obj

    def cleargrads(self): # 파라미터 초기화
        for param in self.params():
            param.cleargrad()

class Model(Layer): #그림 그리는 기능 추가
    def plot(self, *inputs, to_file='model.png'):
        y = self.forward(*inputs)
        return utils.plot_dot_graph(y, verbose=True, to_file=to_file) #verbose 가 True면 ndarray 인스턴스의 형상과 타입도 계산 그래프에 표시해줌

class MLP(Model):
    def __init__(self, fc_output_sizes, activation = F.sigmold):
        # fc_output_sizes는 신경망을 구성하는 완전연결계층들의 출력 크기를 튜플 또는 리스트로 지정합니다. 
        # activation은 활성화 함수를 지정합니다
        super().__init__()
        self.activation = activation
        self.layers = []

        for i, out_size in enumerate(fc_output_sizes):
            layer = L.Linear(out_size)
            setattr(self,'l' + str(i), layer) #l1 = L.Linear(out_size) 처럼 객체에 변수 저장중 
            self.layers.append(layer)

    def forward(self,x):
        for i  in self.layers[:-1]:
            x = self.activation(l(x))
        return self.layers[-1](x)


class Linear(Layer): # Layer를 상속받아 사용하는 클래스 예시
    def __init__(self, out_size, nobias=False, dtype=np.float32, in_size=None):
        super().__init__() # 필수다 이거 안해주면 기초 클래스의 생성자를 가져오지 않는다는 건데 보통 기초 클래스에 속성을 정의하니까 그거 통쨰로 안쓴다고 하는 것과 다른게 없다
        self.in_size = in_size
        self.out_size = out_size
        self.dtype = dtype

        self.W = Parameter(None, name='W')
        if self.in_size is not None: # 만일 in_size가 지정되어 있지 않다면 나중으로 연기
            self._init_W()
        if nobias:
            self.b = None
        else:
            self.b = Parameter(np.zeros(out_size, dtype=dtype), name = 'b')

    def _init_W(self):
        I, O = self.in_size, self.out_size
        W_data = np.random(I,O).astype(self.dtype) * np.sprt(1/I)
        self.W.data = W_data

    def forward(self,x):
        if self.W.data is None:
            self.in_size = x.shape[1]
            self._init_W()

        y = F.linear(x, self.W, self.b)
        return y

왜 우리는 Layer 클래스를 만들었을까?

간단하게 말하자면 신경망은 매개변수(경사하강법 등의 최적화 기법에 의해 갱신되는 변수, '가중치'와 '편향'이 이에 해당)를 다뤄야 하는데 

신경망 층이 깊어지면 깊어질수록 매개변수가 어마어마하게 많아질거다

그런데 그 매개변수를 모두 코드에서 따로 관리해주자니 너무 복잡하다.

따라서 매개변수를 담는 구조를 따로 만들어서 매개변수 관리를 자동화 하는 과정을 위해 만든게 Layer 클래스이다.



### 3. pytorch의 module
---

module 클래스를 상속받아 정의할때 유도 클래스는 반드시 __init__과 forward()는 반드시 정의하도록 되어 있다. (당연한 것이 모듈을 받은 모델이 함수처럼 실행될때 forward()를 이용하니까....)

예를 들면 다음과 같이 이용한다

In [1]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

NameError: name 'nn' is not defined

근데 파이토치는 파이토치 나름의 복잡한 시스템을 가진다.
> Module에 관한 파이토치 글 : https://pytorch.org/docs/stable/generated/torch.nn.Module.html

어마어마한 수의 메서드를 같이 상속받지만 보통은 __init__과 forward()만 받는다고 생각하면 편하다.

참고로 반.드.시 super().__init__을 해줘야 하는데 간단하게 Dezero의 예시만 보더라도 self._params = set()을 가져오기 위함이었다.

파이토치는 보다 복잡한 시스템으로 모듈을 만들기 때문에 정상적인 모듈을 만들고 싶다면 그냥 기초클래스의 생성자를 죄다 가져와야 정상적인 모듈이 만들어진다.

참고로 생성자 안가져오면 'AttributeError: cannot assign module before Module.__init__() call' 이란 오류가 뜨면서 제발 가져오라고 한다.



### 4. torch.nn의 구성
---

> https://pytorch.org/docs/stable/nn.html

torch.nn

- Containers

- Convolution Layers

- Pooling layers

- Padding Layers

- Non-linear Activations (weighted sum, nonlinearity)

- Non-linear Activations (other)

- Normalization Layers

- Recurrent Layers

- Transformer Layers

- Linear Layers

- Dropout Layers

- Sparse Layers

- Distance Functions

- Loss Functions

- Vision Layers

- Shuffle Layers

- DataParallel Layers (multi-GPU, distributed)

- Utilities

- Quantized Functions

- Lazy Modules Initialization

### 5. 다양한 Layer
---
1. nn.Flatten

    Flatten 람수는 완전연결계층에 이미지 입력을 위하여 이미지 데이터를 변환해주는 함수이다.

    이미지 데이터는 일반적으로 다차원 데이터로 이루어져 있다. 입력 이미지를 평탄하게 즉, 1차원 배열로 만드는 함수이다.

    API - torch.nn.Flatten(start_dim=1, end_dim=- 1)
    
    + parameters

        - start_dim (int) : 첫번째 축 (default = 1)

        - end_dim (int) : 마지막 축 (default = -1)

    + Examples


In [ ]:
input = torch.randn(32, 1, 5, 5)
# With default parameters
m = nn.Flatten()
output = m(input)
output.size()
# torch.Size([32, 25])
# With non-default parameters
m = nn.Flatten(0, 2)
output = m(input)
output.size()
torch.Size([160, 5])

2. nn.Linear

    Linear 계층 저장된 가중치와 편향을 사용하여 입력에 선형 변환을 적용하는 모듈

    Dezero에서 만든 그 모델과 같은 역할을 한다. 다만 다른것은 Dezero의 Linear는 입력값을 딱히 안줘도 호출되는 순간 자동으로 입력값을 설정하게끔 했지만 파이토치에는 그 기능이 없다.

    API - torch.nn.Linear(in_features, out_features, bias=True, device=None, dtype=None)
    
    + parameters

        - in_features (int) – 입력 셈플의 크기

        - out_features (int) – 출력 샘플의 크기

        - bias (bool) - False로 설정하면 레이어는 가산 편향을 학습하지 않습니다. Default : True

    + examples


In [ ]:
m = nn.Linear(20, 30) # 가중치를 (20 x 30)으로 설정해서 (a x 20)이 들어오면 가중치를 곱해 (a x 30)으로 변환함
input = torch.randn(128, 20)
output = m(input)
print(output.size()) # torch.Size([128, 30])

3. nn.Sequential

    > nn.Sequential 은 순서를 갖는 모듈의 컨테이너입니다.  
    > 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달됩니다.  
    > 순차 컨테이너(sequential container)를 사용하여 아래의 seq_modules 와 같은 신경망을 빠르게 만들 수 있습니다.  

    방금전까지만 해도 '아 내가 따로 클래스 정의해서 레이어를 모아야 하나?' 라는 고민을 했는데 바로 해결되었다

    API - torch.nn.Sequential(*args: Module)
        - torch.nn.Sequential(arg: OrderedDict[str, Module])

    + Example

In [ ]:
model = nn.Sequential(
          nn.Conv2d(1,20,5),
          nn.ReLU(),
          nn.Conv2d(20,64,5),
          nn.ReLU()
        )

model = nn.Sequential(OrderedDict([
          ('conv1', nn.Conv2d(1,20,5)),
          ('relu1', nn.ReLU()),
          ('conv2', nn.Conv2d(20,64,5)),
          ('relu2', nn.ReLU())
        ]))

### 6. 대단한 착각을 하시고 계시군요
---
지금 정리해둔게 완전 엉망이다...

자 정리하겠다.

계층은 계산 그래프(매개변수와 함수를 활용한)와 매개변수를 품고 있는 것이다.

Dezero에서 Layer라는 클래스를 만들어 규칙에 맞춰서 다양한 계층을 생성할 수 있다. 

근데 Dezero에서 계층에서 그림 그리는 기능만 추가했더니 그걸 신경망 모델의 틀이라고 하고 있다.

그렇다!

계층이라고 했긴 했는데, 신경망 모델 또한 계층인 것이다.(!!!)

계층에 다양한 계층을 넣어서 더 큰 계층을 만들고 그것을 신경망이라고 할 수 있는것!

그렇기에 신경망을 제작하는 것과 계층을 만드는 방법은 다른게 없다.

물론 신경망과 계층은 서로 구현 형식이 살짝 다르다.

계층은 파라미터(매개변수)를 관리하기 위해서 계산그래프와 파라미터가 있는 것이고

신경망은 파라미터가 없다

딱 그 정도 차이이다. 

![](./%EC%9D%B4%EB%AF%B8%EC%A7%80/image_21.jpg)

수식(함수)라고 했는데 이미 텐서에서 사용하는 +, - , *, / 는 이미 Function이라는 클래스를 상속받아서 순전파와 역전파가 있는 클래스이다. 즉, 보통의 연산자가 아니란 것

그렇기 때문에 계층에서 따로 역전파(backward)를 정의하지 않아도 자동적으로 역전파가 흘러간다. 순전파만 형성해도 알아서 역전파가 흘러갈 것이란 뜻(순전파를 흘려야 계산그래프가 형성되기 때문...)